In [4]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 500
crossover_prob = 1.0
mutation_prob = 0.05
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%2==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    size = len(parent1)
    p1, p2 = [0] * size, [0] * size

    # Select two random points in the chromosomes
    point1 = random.randint(0, size - 1)
    point2 = random.randint(0, size - 1)

    # Ensure the two points are not the same
    while point1 == point2:
        point2 = random.randint(0, size - 1)

    # Order the points
    if point1 > point2:
        point1, point2 = point2, point1

    # Copy the respective parts from the parents to the children
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    rand_tmp = random.random()
    rand_idx = random.randint(1,len(individual))
    if rand_tmp < mutation_prob:
        for i in range(0,len(individual), rand_idx):
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8358409940844722
Generation 1 Avg: 0.41712076937329307
Generation 2 Max: 0.8358409940844722
Generation 2 Avg: 0.42491177030255217
Generation 3 Max: 0.8358409940844722
Generation 3 Avg: 0.4285084004692789
Generation 4 Max: 0.8358409940844722
Generation 4 Avg: 0.436875003309468
Generation 5 Max: 0.8358409940844722
Generation 5 Avg: 0.44435253191739094
Generation 6 Max: 0.8358409940844722
Generation 6 Avg: 0.45207813153514786
Generation 7 Max: 0.8358409940844722
Generation 7 Avg: 0.4556747339522439
Generation 8 Max: 0.8358409940844722
Generation 8 Avg: 0.4586330682797877
Generation 9 Max: 0.8358409940844722
Generation 9 Avg: 0.4668504289590857
Generation 10 Max: 0.8358409940844722
Generation 10 Avg: 0.4738031508053854
Generation 11 Max: 0.8358409940844722
Generation 11 Avg: 0.48184383256793956
Generation 12 Max: 0.8358409940844722
Generation 12 Avg: 0.4852668803838413
Generation 13 Max: 0.8358409940844722
Generation 13 Avg: 0.49332902639884024
Generation 14 Max: 0.8358

Generation 109 Max: 0.8358409940844722
Generation 109 Avg: 0.8247914183937199
Generation 110 Max: 0.8358409940844722
Generation 110 Avg: 0.8244988610976135
Generation 111 Max: 0.8358409940844722
Generation 111 Avg: 0.8248824762452798
Generation 112 Max: 0.8358409940844722
Generation 112 Avg: 0.824856156043174
Generation 113 Max: 0.8358409940844722
Generation 113 Avg: 0.8249418986238989
Generation 114 Max: 0.8358409940844722
Generation 114 Avg: 0.825042280620777
Generation 115 Max: 0.8358409940844722
Generation 115 Avg: 0.8244960081465742
Generation 116 Max: 0.8358409940844722
Generation 116 Avg: 0.8238201587333568
Generation 117 Max: 0.8358409940844722
Generation 117 Avg: 0.8255520744218267
Generation 118 Max: 0.8358409940844722
Generation 118 Avg: 0.8260960194722254
Generation 119 Max: 0.8358409940844722
Generation 119 Avg: 0.8263583090851709
Generation 120 Max: 0.8358409940844722
Generation 120 Avg: 0.8266588090525867
Generation 121 Max: 0.8358409940844722
Generation 121 Avg: 0.82666

Generation 215 Max: 0.8358409940844722
Generation 215 Avg: 0.8327444037767673
Generation 216 Max: 0.8358409940844722
Generation 216 Avg: 0.8323385754201782
Generation 217 Max: 0.8358409940844722
Generation 217 Avg: 0.8325946401983412
Generation 218 Max: 0.8358409940844722
Generation 218 Avg: 0.8320993780132823
Generation 219 Max: 0.8358409940844722
Generation 219 Avg: 0.8312750997767857
Generation 220 Max: 0.8358409940844722
Generation 220 Avg: 0.8321462978090584
Generation 221 Max: 0.8358409940844722
Generation 221 Avg: 0.8323414998767314
Generation 222 Max: 0.8358409940844722
Generation 222 Avg: 0.8322874347352256
Generation 223 Max: 0.8358409940844722
Generation 223 Avg: 0.832166363674075
Generation 224 Max: 0.8358409940844722
Generation 224 Avg: 0.8317629738679396
Generation 225 Max: 0.8358409940844722
Generation 225 Avg: 0.8324033965259677
Generation 226 Max: 0.8358409940844722
Generation 226 Avg: 0.8324926574416286
Generation 227 Max: 0.8358409940844722
Generation 227 Avg: 0.8323

Generation 321 Max: 0.8358409940844722
Generation 321 Avg: 0.8335556568767614
Generation 322 Max: 0.8358409940844722
Generation 322 Avg: 0.8341079100082645
Generation 323 Max: 0.8358409940844722
Generation 323 Avg: 0.8334484838110212
Generation 324 Max: 0.8358409940844722
Generation 324 Avg: 0.8340931069677141
Generation 325 Max: 0.8358409940844722
Generation 325 Avg: 0.834163131150605
Generation 326 Max: 0.8358409940844722
Generation 326 Avg: 0.8342051510218321
Generation 327 Max: 0.8358409940844722
Generation 327 Avg: 0.8338751618117364
Generation 328 Max: 0.8358409940844722
Generation 328 Avg: 0.8335942323818825
Generation 329 Max: 0.8358409940844722
Generation 329 Avg: 0.8331198461179219
Generation 330 Max: 0.8358409940844722
Generation 330 Avg: 0.832588939772652
Generation 331 Max: 0.8358409940844722
Generation 331 Avg: 0.8322319513716403
Generation 332 Max: 0.8358409940844722
Generation 332 Avg: 0.8341354894880634
Generation 333 Max: 0.8358409940844722
Generation 333 Avg: 0.83264

Generation 427 Max: 0.8358409940844722
Generation 427 Avg: 0.8344962465047107
Generation 428 Max: 0.8358409940844722
Generation 428 Avg: 0.8338268640482659
Generation 429 Max: 0.8358409940844722
Generation 429 Avg: 0.8343067375093876
Generation 430 Max: 0.8358409940844722
Generation 430 Avg: 0.8335306955138425
Generation 431 Max: 0.8358409940844722
Generation 431 Avg: 0.8326480663250839
Generation 432 Max: 0.8358409940844722
Generation 432 Avg: 0.8340492550110562
Generation 433 Max: 0.8358409940844722
Generation 433 Avg: 0.8343569414260239
Generation 434 Max: 0.8358409940844722
Generation 434 Avg: 0.8343452062669795
Generation 435 Max: 0.8358409940844722
Generation 435 Avg: 0.8346338864231619
Generation 436 Max: 0.8358409940844722
Generation 436 Avg: 0.8339216044378179
Generation 437 Max: 0.8358409940844722
Generation 437 Avg: 0.8344592579936289
Generation 438 Max: 0.8358409940844722
Generation 438 Avg: 0.8344252063009566
Generation 439 Max: 0.8358409940844722
Generation 439 Avg: 0.834